# Extensions for FluentAssertions


## EventualAssertions

The extension allows to loop FluentAssertions checks for a specified period of time. The extension relies heavily on the [Assertion Scopes](https://fluentassertions.com/introduction#assertion-scopes) feature and is an assertion scope itself. Any failed assertion among those wrapped with the extension will cause another attempt. The last try is always done out of the Assertion Scope (under the parent scope)

A particular goal when designing the extension was to provide the experience similar to given by Assertion Scopes, that is, no additional nesting would be needed. This is elegantly achievable in F\#, but a little bit of a challenge for C\#

### EventualAssertions in C\#

In [ ]:
#r "nuget: mazharenko.FluentAssertions.Extensions, *"

Installed Packages mazharenko.FluentAssertions.Extensions, 2.0.0

In [ ]:
using mazharenko.FluentAssertions.Extensions;
using FluentAssertions;
using FluentAssertions.Extensions;

var attempts = EventualAssertions.Attempts(4.Seconds(), 400.Milliseconds());

`attempts` now is of a type that implements `IEnumerable` returning `IEnumerator` wrapping an assertion scope. On each new element requested the assertion scope is checked and the specified delay is waited. Here, assertions are supposed to be placed in the body of the loop over `attempts`. Elements of `attempts` can provide some meta information about the attempt.

In [ ]:
#!time
var inFuture = DateTime.Now.AddSeconds(3);

foreach (var attempt in attempts)
{
    Console.WriteLine(attempt.Number);
    DateTime.Now.Should().BeAfter(inFuture);
}

1
2
3
4
5
6
7
8
9


Wall time: 3579.3668ms

In [ ]:
foreach (var _ in EventualAssertions.Attempts(400, 50))
    (1+1).Should().Be(3);

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in /_/Src/FluentAssertions/Execution/DefaultAssertionStrategy.cs:line 25
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in /_/Src/FluentAssertions/Execution/AssertionScope.cs:line 274
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in /_/Src/FluentAssertions/Execution/AssertionScope.cs:line 246
   at FluentAssertions.Execution.AssertionScope.FailWith(String message, Object[] args) in /_/Src/FluentAssertions/Execution/AssertionScope.cs:line 296
 

EventualAssertions can be nested in other assertion scopes.

In [ ]:
using FluentAssertions.Execution;

using (new AssertionScope())
{
    foreach (var _ in EventualAssertions.Attempts(400, 50))
    {
        (1+1).Should().Be(3);
        break;
    }
    foreach (var _ in EventualAssertions.Attempts(400, 50))
    {
        (2*2).Should().Be(5);
        break;
    }
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context) in /_/Src/FluentAssertions/Execution/CollectingAssertionStrategy.cs:line 42
   at Submission#5.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Likewise, other assertion scopes can be nested in EventualAssertions

In [ ]:
foreach (var _ in EventualAssertions.Attempts(400, 30))
{
    using (new AssertionScope()) 
    {
        (1+1).Should().Be(3);
        (2*2).Should().Be(5);
    }
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context) in /_/Src/FluentAssertions/Execution/CollectingAssertionStrategy.cs:line 42
   at Submission#19.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Asynchronous methods are also available. Those would use `Task.Delay` instead of `Thread.Sleep`.

In [ ]:
#!time
var inFuture = DateTime.Now.AddSeconds(3);
await foreach(var _ in EventualAssertions.AttemptsAsync(4.Seconds(), 400.Milliseconds()))
{
    DateTime.Now.Should().BeAfter(inFuture);
}

Wall time: 3361.0724ms

### EventualAssertions in F\#

The F\# implementation provides a special looping computation expression builder.

In [ ]:
#r "nuget: mazharenko.FluentAssertions.Extensions.FSharp, *"

open mazharenko.FluentAssertions.Extensions.EventualAssertions

Installed Packages mazharenko.FluentAssertions.Extensions.FSharp, 2.0.0

In [ ]:
#!time

open FluentAssertions;

let inFuture = DateTime.Now.AddSeconds(float 3)
(eventually (TimeSpan.FromSeconds(4)) (TimeSpan.FromMilliseconds(40))) {
    DateTime.Now.Should().BeAfter(inFuture, null, null) |> ignore;
}

Wall time: 3956.1151ms

In [ ]:
(eventuallyMs 400 40) {
    (1+1).Should().Be(3, null, null) |> ignore;
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in /_/Src/FluentAssertions/Execution/DefaultAssertionStrategy.cs:line 25
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in /_/Src/FluentAssertions/Execution/AssertionScope.cs:line 285
   at FluentAssertions.Numeric.NumericAssertions`2.Be(T expected, String because, Object[] becauseArgs) in /_/Src/FluentAssertions/Numeric/NumericAssertions.cs:line 60
   at FSI_0007.it@2-1.Invoke(Unit unitVar)
   at mazharenko.FluentAssertions.Extensions.EventualAssertions.EventualAssertionsBuil

EventualAssertions builder can be nested in other assertion scopes.

In [ ]:
open FluentAssertions.Execution;

using (new AssertionScope()) (fun _ -> 
    (eventuallyMs 0 0) {
      (1+1).Should().Be(3, null, null) |> ignore;
    }
    (eventuallyMs 0 0) {
      (2*2).Should().Be(5, null, null) |> ignore;
    }
)

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context) in /_/Src/FluentAssertions/Execution/CollectingAssertionStrategy.cs:line 42
   at Microsoft.FSharp.Core.Operators.Using[T,TResult](T resource, FSharpFunc`2 action) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4887
   at <StartupCode$FSI_0029>.$FSI_0029.main@()

Likewise, other assertion scopes can be nested in EventualAssertions

In [ ]:
open FluentAssertions.Execution;

(eventuallyMs 400 40) {
    using (new AssertionScope()) (fun _ -> 
        (1+1).Should().Be(3, null, null) |> ignore;
        (2*2).Should().Be(5, null, null) |> ignore;
    )
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context) in /_/Src/FluentAssertions/Execution/CollectingAssertionStrategy.cs:line 42
   at Microsoft.FSharp.Core.Operators.Using[T,TResult](T resource, FSharpFunc`2 action) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4887
   at FSI_0041.it@4-25.Invoke(Unit unitVar)
   at mazharenko.FluentAssertions.Extensions.EventualAssertions.EventualAssertionsBuilder.Run(FSharpFunc`2 f)
   at <StartupCode$FSI_0041>.$FSI_0041.main@()

Asynchronous methods are also available. Those would use `Async.Sleep` instead of `Thread.Sleep`.

In [ ]:
#!time

let inFuture = DateTime.Now.AddSeconds(3)
(eventuallyAsyncMs 4000 400) {
    DateTime.Now.Should().BeAfter(inFuture, null, null) |> ignore
} |> Async.RunSynchronously

Wall time: 3322.205ms